In [1]:
from IPython import display

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time
!pip install torch
import torch

import h5py
from IPython.display import clear_output

import math

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


hey this is ideal


#Load h5 data

In [5]:
df = h5py.File("/content/drive/MyDrive/data/neutrino_data/neutrino1.h5",'r')

#Get an numpy array containing the event image, and reshape it from flat to 2x100x80
images = []

print(np.shape(df['cvnmap']))
n = 0
for i in range(0,math.floor(len(df['cvnmap']))):
  event0=np.array(df['cvnmap'][i]).reshape((2,100,80))
  n+=1
  clear_output()
  print(str(100*round(n/math.floor(len(df['cvnmap'])),2)) + "%")
  images.append(event0[1].T)

images = np.array(images)
print(images.shape)

OSError: ignored

In [ ]:
train_images_unprocessed = np.array(images[:-math.floor(len(images)*0.2)])           #torch.tensor(np.array())
test_images_unprocessed  = np.array(images[len(images) - math.floor(len(images)*0.2):])

print(train_images_unprocessed.shape)
print(test_images_unprocessed.shape)

#Load personal dataset

In [ ]:
def convertImagesToArray(imageptrs):
  images = []

  n = 0
  for pImage in imageptrs:
    #print(plt.imread(pImage[:-1]))
    images.append(plt.imread(pImage[:-1]))
    n+=1
    clear_output()
    print(str(100 * round(n/len(imageptrs),2)) + "%")

  return images

In [ ]:
file = open("/content/drive/MyDrive/Testing_VAE/pointerlists/stringlist.txt", 'r')

imageptrs = []
#print(len(file.readlines()))
for line in file.readlines():
  imageptrs.append(line)

train_images_unprocessed = torch.tensor(np.array(convertImagesToArray(imageptrs[:-round(len(imageptrs)*0.2)])))
test_images_unprocessed  = torch.tensor(np.array(convertImagesToArray(imageptrs[-round(len(imageptrs)*0.2):-1])))
print(train_images_unprocessed.shape)
print(test_images_unprocessed.shape)

100.0%
torch.Size([1136, 128, 128, 3])
torch.Size([283, 128, 128, 3])


#Preprocessing functions

Define preprocessing procedure

In [ ]:
def preprocess_images(imagelist):
  imagelist = imagelist.reshape((imagelist.shape[0], imagelist.shape[1], imagelist.shape[2], 3)) / torch.max(imagelist)
  #imagelist.reshape((imagelist.shape[0], imagelist.shape[1], imagelist.shape[2], 1))
  return tf.cast(imagelist, tf.float32) #np.where(imagelist > .5, 1.0, 0.0).astype('float32')

def preprocess_images_reduce(imagelist):
  #print(imagelist.shape)
  imagelist_processed = []
  for i in range(len(imagelist)):
    img = imagelist[i]
    cropped_img = 255.*img[:,:80]/np.amax(img)
    imagelist_processed.append(cropped_img)
  imagelist_processed = np.array(imagelist_processed)
  #print(imagelist_processed.shape)
  imagelist_processed = torch.tensor(imagelist_processed)
  imagelist_processed = imagelist_processed.reshape((imagelist_processed.shape[0], imagelist_processed.shape[1], imagelist_processed.shape[2], 1))
  return tf.cast(imagelist_processed, tf.float32)
    

#Preprocess using normalization of image brightness

In [ ]:
train_images = preprocess_images(train_images_unprocessed)
test_images  = preprocess_images( test_images_unprocessed)

print(train_images.shape)
print(test_images.shape)

ValueError: ignored

#Preprocess NEUTRINO images to be square and normalized

In [ ]:
train_images = preprocess_images_reduce(train_images_unprocessed)
test_images  = preprocess_images_reduce( test_images_unprocessed)

print(train_images.shape)
print(test_images.shape)

Training/Testing Variables

In [ ]:
train_size = len(train_images)
batch_size = 256
test_size  = len(test_images) 

Batch/Dataset construction

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices(train_images)
                 .shuffle(train_size).batch(batch_size))
test_dataset = (tf.data.Dataset.from_tensor_slices(test_images)
                .shuffle(test_size).batch(batch_size))

#CVAE Network (Encoder + Decoder) for AURORA data

In [ ]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(128, 128, 3)),
            tf.keras.layers.Reshape(target_shape=(128,128,3)),
            tf.keras.layers.Conv2DTranspose(
                filters=4, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2D(
                filters=4, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=8, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=16, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.AvgPool2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.AvgPool2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
         
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=4*4*batch_size, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(4,4,batch_size)),

            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=16, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=8, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),

            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=3, kernel_size=3, strides=(2,2), padding='same')
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

  def call(self, inputs):
    return self.decode(self.reparameterize(self.encode(inputs)))

#CVAE for neutrino data

In [ ]:
class CVAENeutrino(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAENeutrino, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(80, 80, 1)),
            tf.keras.layers.Conv2DTranspose(
                filters=4, kernel_size=3, strides=(4,4), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=4, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2D(
                filters=4, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=8, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=16, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.AvgPool2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.AvgPool2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
         
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=5*5*batch_size, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(5,5,batch_size)),

            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=16, kernel_size=3, strides=(2,2), padding='same',
                activation='relu'),

            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=(2,2), padding='same')
        ]
    )

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x):
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

  def call(input):
    return self.decode(self.encode(input))

#Loss function + training step methods

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x)
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#Setup for training

In [ ]:
train_size = len(train_images)
batch_size = 128
test_size  = len(test_images)
train_dataset = (tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_size).batch(batch_size))
test_dataset  = (tf.data.Dataset.from_tensor_slices(test_images).shuffle(test_size).batch(batch_size))

epochs = 1000
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 25
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
random_vector_for_generation = tf.random.normal(
    shape=[num_examples_to_generate, latent_dim])
model = CVAENeutrino(latent_dim)

#Taking a sample and epoch, make and show the images the model predicted

In [ ]:
def generate_and_save_images(model, epoch, test_sample):
  mean, logvar = model.encode(test_sample)
  z = model.reparameterize(mean, logvar)
  predictions = model.sample(z)

  fig1 = plt.figure(figsize=(4, 4))
  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i + 1)
    plt.imshow(test_sample[i, :, :, 0])
    plt.axis('off')

  fig2 = plt.figure(figsize=(4, 4))
  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow(predictions[i, :, :, 0])#, cmap='gray') 
    plt.axis('off')

  # tight_layout minimizes the overlap between 2 sub-plots
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

#Pick a sample of the test set for generating output images

In [ ]:
# Pick a sample of the test set for generating output images
assert batch_size >= num_examples_to_generate
for test_batch in test_dataset.take(1):
  test_sample = test_batch[0:num_examples_to_generate, :, :]

#Using a sample, make a set of prediction images

In [3]:
generate_and_save_images(model, 0, test_sample)

for epoch in range(1, epochs + 1):
  start_time = time.time()
  for train_x in train_dataset:
    train_step(model, train_x, optimizer)
  end_time = time.time()

  loss = tf.keras.metrics.Mean()
  for test_x in test_dataset:
    loss(compute_loss(model, test_x))
  elbo = -loss.result()
  print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
        .format(epoch, elbo, end_time - start_time))
  if epoch%30 == 0:
    display.clear_output(wait=False)
    generate_and_save_images(model, epoch, test_sample)
    

NameError: ignored

In [ ]:
anim_file = 'cvae.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  n = 0
  for filename in filenames:
    if n%1==0:
      image = imageio.imread(filename)
      writer.append_data(image)
    n = n + 1
  image = imageio.imread(filename)
  writer.append_data(image)

#Saving the model

In [ ]:
model_name = "modelTEST"
model.save_weights("/content/drive/MyDrive/Testing_VAE/model_checkpoints/"+model_name)